In [1]:
# Task: Text Classification: classify the effect on members in 34 classes (No labels available)
# "Unsupervised classification" using informative text class labels as class prototypes in a bilingual vector space
# AI4Good Hackathon
# Alex Merritt
# Borrows from: https://github.com/JairParra/NLPChallengeAI4Good/blob/master/tools/spaCy_tools%20_Jair_Copy.ipynb
# Requires: https://github.com/Babylonpartners/fastText_multilingual
# and https://fasttext.cc/docs/en/crawl-vectors.html
# and https://pypi.org/project/langdetect/


In [28]:
from fasttext import FastVector
from langdetect import detect
import spacy 
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

In [29]:
french_stopwords = stopwords.words('french')

nlp = spacy.load("fr_core_news_md")

fr_dictionary = FastVector(vector_file='cc.fr.300.vec')

fr_dictionary.apply_transform('alignment_matrices/fr.txt')

reading word vectors from cc.fr.300.vec


In [2]:
en_dictionary = FastVector(vector_file='cc.en.300.vec')
en_dictionary.apply_transform('alignment_matrices/en.txt')

reading word vectors from cc.en.300.vec


KeyboardInterrupt: 

In [30]:
df = pd.read_excel(open('Baseresultats.xlsx', 'rb'),
              sheet_name='Sheet1', header=None)
indicators = df[0].tolist()
indicators = [x.strip() for x in indicators]

print(indicators)

['Autonomie', "Bris de l'isolement", 'Communication', 'Compétences', 'Confiance en soi', 'Connaissance de soi', 'Connaissances', 'Conscientisation / Esprit critique', "Développement (de l'enfant)", 'Développement de pratiques démocratiques', 'Empowerment collectif', 'Empowerment individuel', 'Estime de soi', 'Habiletés cognitives', 'Habiletés dans la vie quotidienne', 'Habiletés sociales', 'Habitudes de vie', 'Identification des besoins', 'Intégration sociale', 'Lien de confiance', 'Liens familiaux', 'Mixité sociale et culturelle', 'Participation citoyenne', 'Plaisir', 'Prise de parole', 'Réciprocité', 'Répit', "Réseau d'entraide", 'Résultats scolaires', 'Sécurité', "Sentiment d'appartenance", 'Sentiment de valorisation', 'Socialisation', 'Soutien']


In [31]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / n1 / n2

In [32]:
def vectorByLanguage(word, lang):

    try:
        if(lang == 'en'):
            return en_dictionary[word]
        elif(lang == 'fr'):
            return fr_dictionary[word]
        else:
            print("Found a non-English, non-French doc. Language detected : " + lang)
    except:
        return np.zeros(300)

def docIndicatorSimilarity(indicator, row, doc_lang): 
    tags = ["DET","ADP","PUNCT","CONJ","CCONJ","NUM","SYM","SPACE"]
    indicator_words = indicator.split(' ')
    indicator_word_vecs = [vectorByLanguage(word, 'fr') for word in indicator_words]
    indicator_avg_vec = np.mean(indicator_word_vecs, axis=0)

    doc = str(row['RESULTATS_2018'])
    
    
    sent_toks = [sent for sent in nlp(doc).sents]

    flt_sent_toks = [[token for token in toks if token.pos_ not in tags and token.text.isalpha()
                     and token.text not in french_stopwords] for toks in sent_toks]
    
    flat_flt_sent_toks = [item.text for sublist in flt_sent_toks for item in sublist]
    doc_words = doc.split(' ')

    doc_word_vecs = [vectorByLanguage(word, doc_lang) for word in flat_flt_sent_toks]
    doc_avg_vec = np.mean(doc_word_vecs, axis=0)
    return similarity(indicator_avg_vec, doc_avg_vec)

In [35]:
d = {}

results_df = pd.read_excel(open('Baseresultats.xlsx', 'rb'), sheet_name='Liste brute')
for index, row in results_df.iterrows():
    doc_lang = detect(str(row['RESULTATS_2018']))
    if ('Effet' in str(row['Type de résultat']) and (doc_lang == 'en' or doc_lang == 'fr')):
            similarities = [docIndicatorSimilarity(indicator, row, doc_lang) for indicator in indicators]       
            #print(similarities)
            max_similarity = np.argmax(similarities)
            max_similarities = np.argsort(similarities)[-5:]
            #max_similarities = max_similarities.reverse()
            
            #print(max_similarities)
            d[row['NO']] = [indicators[max_similarities[i]] for i in range(5)]
            print("ID: " + str(row['NO']) + " " + str(d[row['NO']]))


ID: 7848 ['Développement de pratiques démocratiques', 'Mixité sociale et culturelle', 'Intégration sociale', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 193624 ['Connaissance de soi', 'Mixité sociale et culturelle', 'Habitudes de vie', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 163221 ['Habitudes de vie', 'Estime de soi', 'Identification des besoins', 'Connaissance de soi', 'Habiletés dans la vie quotidienne']
ID: 139813 ['Habiletés dans la vie quotidienne', 'Intégration sociale', 'Habiletés cognitives', 'Habiletés sociales', 'Identification des besoins']
ID: 118-09 ['Connaissance de soi', 'Intégration sociale', 'Habitudes de vie', 'Identification des besoins', 'Habiletés dans la vie quotidienne']
ID: 9222 ['Habitudes de vie', 'Mixité sociale et culturelle', 'Habiletés dans la vie quotidienne', 'Résultats scolaires', 'Identification des besoins']
ID: 103 ['Estime de soi', 'Habitudes de vie', 'Connaissance de soi', 'Habiletés dans l

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


ID: 7930 ['Estime de soi', 'Habiletés cognitives', 'Habiletés dans la vie quotidienne', "Développement (de l'enfant)", 'Soutien']
ID: 7810 ['Habitudes de vie', 'Mixité sociale et culturelle', 'Habiletés dans la vie quotidienne', 'Intégration sociale', 'Identification des besoins']
ID: 8993 ['Connaissance de soi', 'Habitudes de vie', 'Mixité sociale et culturelle', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 7874 ['Habitudes de vie', 'Confiance en soi', 'Connaissance de soi', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 6608 ['Lien de confiance', 'Mixité sociale et culturelle', 'Connaissance de soi', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 135613 ['Habitudes de vie', 'Estime de soi', 'Identification des besoins', 'Connaissance de soi', 'Habiletés dans la vie quotidienne']
ID: 9532 ['Mixité sociale et culturelle', 'Connaissance de soi', 'Habitudes de vie', 'Habiletés dans la vie quotidienne', 'Identificat

ID: 6612 ['Confiance en soi', 'Estime de soi', 'Connaissance de soi', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 138506 ['Résultats scolaires', 'Connaissance de soi', 'Habiletés dans la vie quotidienne', 'Habitudes de vie', 'Identification des besoins']
ID: 4213 ['Confiance en soi', 'Connaissance de soi', 'Habitudes de vie', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 328045 ['Estime de soi', 'Habiletés dans la vie quotidienne', 'Mixité sociale et culturelle', 'Habitudes de vie', 'Connaissance de soi']
ID: 1012 ['Mixité sociale et culturelle', 'Habitudes de vie', 'Connaissance de soi', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 1013 ['Habitudes de vie', 'Mixité sociale et culturelle', 'Habiletés dans la vie quotidienne', 'Identification des besoins', 'Intégration sociale']
ID: 273241 ['Mixité sociale et culturelle', 'Habiletés dans la vie quotidienne', 'Intégration sociale', 'Habiletés sociales', 'Identi

ID: 8654 ['Estime de soi', 'Habitudes de vie', 'Connaissance de soi', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 7878 ['Habitudes de vie', 'Confiance en soi', 'Mixité sociale et culturelle', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 3147 ['Habiletés dans la vie quotidienne', 'Habitudes de vie', 'Estime de soi', 'Connaissance de soi', 'Identification des besoins']
ID: 118 ['Estime de soi', 'Habitudes de vie', 'Connaissance de soi', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 164826 ['Résultats scolaires', 'Confiance en soi', 'Connaissance de soi', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 7826 ['Connaissance de soi', 'Habitudes de vie', 'Résultats scolaires', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 135633 ['Habitudes de vie', 'Connaissance de soi', 'Mixité sociale et culturelle', 'Habiletés dans la vie quotidienne', 'Identification des besoins']

ID: 278352 ['Estime de soi', 'Mixité sociale et culturelle', 'Connaissance de soi', 'Habiletés dans la vie quotidienne', 'Identification des besoins']
ID: 9588 ['Estime de soi', 'Habiletés cognitives', 'Habiletés dans la vie quotidienne', "Développement (de l'enfant)", 'Soutien']
ID: 715-03 ['Estime de soi', 'Développement de pratiques démocratiques', 'Lien de confiance', 'Connaissance de soi', 'Identification des besoins']
ID: 162996 ['Habiletés dans la vie quotidienne', 'Mixité sociale et culturelle', 'Connaissance de soi', 'Habitudes de vie', 'Identification des besoins']
ID: 7830 ['Mixité sociale et culturelle', 'Estime de soi', 'Habitudes de vie', 'Connaissance de soi', 'Identification des besoins']
ID: 141087 ['Habitudes de vie', 'Habiletés dans la vie quotidienne', 'Résultats scolaires', 'Liens familiaux', 'Identification des besoins']
ID: 715-02 ['Habitudes de vie', 'Habiletés dans la vie quotidienne', 'Lien de confiance', 'Connaissance de soi', 'Identification des besoins']
ID